In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import curve_fit
from numpy import exp
from astropy.modeling import models, fitting
from scipy.optimize import leastsq,curve_fit
from scipy import ndimage
from scipy import asarray as ar,exp
from muse import *

In [2]:
hdu = fits.open('HE0232-0900.wo_absorption.fits')
qso_data = hdu[0].data
qso_error = hdu[1].data
qso_header = hdu[0].header
wavestart = qso_header['CRVAL3']
wavint = qso_header['CD3_3']
wave = wavestart+np.arange(qso_data.shape[0])*wavint 

In [3]:
x = wave
y = qso_data[:,51,67] # I set the central position manual here, but one can also search for maximum light in principle
y_err = qso_error[:,51,67]

In [4]:
z = 0.043143
k = z+1
select = (x>4750*k) & (x<5090*k)

In [5]:
hdu = fits.open('HE0232-0900_central_fit2.fits')
central_tab = hdu[1].data
central_columns = hdu[1].header
vel_OIII1 = central_tab.field('vel_OIII1')[0]
vel_sigma_OIII1 = central_tab.field('vel_sigma_OIII1')[0]
vel_OIII2 = central_tab.field('vel_OIII2')[0]
vel_sigma_OIII2 = central_tab.field('vel_sigma_OIII2')[0]
vel_OIII_br = central_tab.field('vel_OIII_br')[0]
vel_sigma_OIII_br = central_tab.field('vel_sigma_OIII_br')[0]
vel_Hb1 = central_tab.field('vel_Hb1')[0]
vel_Hb2 = central_tab.field('vel_Hb2')[0]
vel_sigma_Hb1 = central_tab.field('vel_sigma_Hb1')[0]
vel_sigma_Hb2 = central_tab.field('vel_sigma_Hb2')[0]
fixed_param = [vel_OIII1,vel_sigma_OIII1,vel_OIII2,vel_sigma_OIII2,vel_OIII_br,vel_sigma_OIII_br,vel_Hb1,vel_sigma_Hb1,vel_Hb2,vel_sigma_Hb2]  


In [12]:
[central_x,central_y]= [67,51]
mini_cube = qso_data[:,central_y - 5:central_y + 6,central_x - 5:central_x + 6]
mini_cube_error = qso_error[:,central_y - 5:central_y + 6,central_x - 5:central_x + 6]
qso_header['CRPIX1'] = qso_header['CRPIX1'] - (central_x - 5)
qso_header['CRPIX2'] = qso_header['CRPIX2'] - (central_y - 5)
new_hdu = fits.HDUList([fits.PrimaryHDU(mini_cube),fits.ImageHDU(mini_cube_error)])
new_hdu[0].header = qso_header

mini_data = mini_cube
mini_error = mini_data

In [13]:
par = np.zeros((12,mini_data.shape[1],mini_data.shape[2]),dtype=np.float32) #dtype = datatype. the type of the output array
err = np.zeros((12,mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
fitted = np.zeros((np.shape(wave[select])[0],mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
residuals = np.zeros((np.shape(wave[select])[0],mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)

In [15]:
for i in range(mini_data.shape[1]):
    for j in range(mini_data.shape[2]):
        y = mini_data[:,i,j][select]
        y_err = mini_error[:,i,j][select]
        x = wave[select]    

        x0 = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,-0.001,0.7]
        popt1,pcov1 = leastsq(full_gauss2_fixkin,x0,args = (x,y,y_err,fixed_param),maxfev=10000000)
        par[:,i,j] = popt1
        model = full_gauss2_fixkin(popt1,x,y,y_err,fixed_param)*(y_err)+y
 
        fitted[:,i,j] = model
        
        residuals = mini_data[:,i,j][select] - fitted[:,i,j]
        
        Monte_Carlo_loops = 5
        parameters_MC = np.zeros((len(popt1),Monte_Carlo_loops))
        
        for l in range(Monte_Carlo_loops):
            iteration_data = np.random.normal(y,y_err) 
            popt_MC,pcov_MC = leastsq(full_gauss2_fixkin,x0=popt1,args=(x,iteration_data,y_err,fixed_param),maxfev = 1000000)
            parameters_MC[:,l]=popt_MC
            
        parameters_err = np.std(parameters_MC,1)
        err[:,i,j]=parameters_err

In [17]:
import random
hdus=[]
hdus.append(fits.PrimaryHDU())
hdus.append(fits.ImageHDU(par[0,:,:],name='amp_Hb_1'))
hdus.append(fits.ImageHDU(par[1,:,:],name='amp_OIII5007_1'))
hdus.append(fits.ImageHDU(par[2,:,:],name='amp_Hb_2'))
hdus.append(fits.ImageHDU(par[3,:,:],name='amp_OIII5007_2'))
hdus.append(fits.ImageHDU(par[4,:,:],name='amp_OIII5007_br'))
hdus.append(fits.ImageHDU(par[5,:,:],name='amp_Hb_br'))
hdus.append(fits.ImageHDU(par[6,:,:],name='amp_Hb1'))
hdus.append(fits.ImageHDU(par[7,:,:],name='amp_Hb2'))
hdus.append(fits.ImageHDU(par[8,:,:],name='amp_Fe5018_1'))
hdus.append(fits.ImageHDU(par[9,:,:],name='amp_Fe5018_2'))
hdus.append(fits.ImageHDU(par[10,:,:],name='m'))
hdus.append(fits.ImageHDU(par[11,:,:],name='c'))
hdu = fits.HDUList(hdus)

hdu.writeto('subcube_HE0232-0900_new.fits',overwrite=True)

In [18]:
hdus=[]
hdus.append(fits.PrimaryHDU())
hdus.append(fits.ImageHDU(err[0,:,:],name='amp_Hb_1'))
hdus.append(fits.ImageHDU(err[1,:,:],name='amp_OIII5007_1'))
hdus.append(fits.ImageHDU(err[2,:,:],name='amp_Hb_2'))
hdus.append(fits.ImageHDU(err[3,:,:],name='amp_OIII5007_2'))
hdus.append(fits.ImageHDU(err[4,:,:],name='amp_OIII5007_br'))
hdus.append(fits.ImageHDU(err[5,:,:],name='amp_Hb_br'))
hdus.append(fits.ImageHDU(err[6,:,:],name='amp_Hb1'))
hdus.append(fits.ImageHDU(err[7,:,:],name='amp_Hb2'))
hdus.append(fits.ImageHDU(err[8,:,:],name='amp_Fe5018_1'))
hdus.append(fits.ImageHDU(err[9,:,:],name='amp_Fe5018_2'))
hdus.append(fits.ImageHDU(err[10,:,:],name='m'))
hdus.append(fits.ImageHDU(err[11,:,:],name='c'))
hdu = fits.HDUList(hdus)

hdu.writeto('subcube_HE0232-0900_err_new.fits',overwrite=True)